In [1]:
#include <iostream>
#include <string>
#include <algoviz/SVG.hpp>

// Unsere eigenen Kreationen a lá Mary Shelley
#include "Vogel.hpp"
#include "Layout.hpp"
#include "Barriere.hpp"

using namespace std;

AlgoViz::clear();

In [2]:
// Spielfeld initialisieren
SVG spielfeld = SVG(800,400);

Layout layout = Layout(spielfeld);
Vogel vogel = Vogel(spielfeld);
Text screenScore;

vogel.hide();

In [3]:
// Benötigte globale Variablen:

string key = "";

int y;

int tmpRunden;
int geschwindigkeit = 0;
int score;
int counter;

Barriere *barriere1;
Barriere *barriere2;

bool reachMid = false;
bool alreadyCrashed = false;
bool ascend = false;

// Eine Variable, um die vertikale Geschwindigkeit zu speichern:
int ascension = 5;

// Startpositionierung
int start = 50;

// Die Iterationen der Hauptschleife unten zählen:
int iteration = 0;



In [4]:
Barriere* erstelleBarriere() {  
    /* Die Position der Öffnung zufällig bestimmen */
    
    int bottleneckY = (rand() % 350);
    return new Barriere(bottleneckY, 100, 800, spielfeld);
}

In [5]:
void vogelAbsturz() {
    
    alreadyCrashed = true;
    key = "x";
    
    // Barrieren transparent stellen    
    barriere1->setFill(0, 0, 0, 0);
    barriere1->setColor(0, 0, 0, 0);
    if (barriere2 != nullptr) {
        barriere2->setFill(0, 0, 0, 0);
        barriere2->setColor(0, 0, 0, 0);
    }
            
    layout.flackern();
    layout.gameOver();
    vogel.death();
}

In [6]:
bool vogelKollidiertMitBarriere (Barriere *b) {
    // Die x-Koordinate des Vogels ist konstant 200.
    int barriere_x = b->getXCoord();
    int oeffnung_y = b->getUpperHeight();
    int oeffnung_h = b->getBottleneckHeight();

    if (barriere_x <= 200 && barriere_x > 150) {
        if (vogel.getYCoord() < oeffnung_y || vogel.getYCoord() > oeffnung_y + oeffnung_h) {
            return true;
        }
    }
    
    return false;
}

In [7]:
bool vogelKollidiert() {
    if (vogelKollidiertMitBarriere(barriere1)) {
        return true;
    }
    
    if (barriere2 != nullptr && vogelKollidiertMitBarriere(barriere2)) {
        return true;
    }
    
    // Überprüft die obere und untere Bildschirmgrenze:
    if (!alreadyCrashed && (vogel.getYCoord() + 20 > 400 || vogel.getYCoord() < 0)) {
        return true;
    }
    
    return false;
}

In [8]:
// Wartet 3 Sekunden und fügt dann den Vogel ins Feld ein.
AlgoViz::sleep(3000);

layout.hideWelcome();
vogel.show();

barriere1 = erstelleBarriere();

In [9]:
// Unsere Hauptschleife
while (key != "x" ) {
    /* Verarbeiten von Tastatureingaben; mit der ↑-Taste wird der Vogel nach oben beschleunigt,
       mit 'x' wird das Spiel beendet. */
    key = spielfeld.lastKey();
 
    if (key == "ArrowUp") {
        // Default Sprunghöhe ist -30 Pixel
        y += -30;
        ascend = true; 
    }

    /* Auf Kollision überprüfen */
    if (vogelKollidiert()) {
        vogelAbsturz();
    } else {
        score++;
    }
    
    /* Vertikale Geschwindigkeit verarbeiten */
    if (ascend) {
        vogel.move(y);
        ascension += -10;
        y += ascension;
        if (ascension <= -10) {
            ascend = false;
        }
    } else {
        // Erdanziehung: Die Beschleunigungsvariable wird um 3 erhöht, 9 ist der Schwellenwert
        if (ascension <= 9) {
            ascension += 3;
        }
        y += ascension;
        vogel.move(y);
    }
    
    // Rotiere den Vogel basierend auf seiner Steigung
    if (ascension < 0) { 
        vogel.rotate(-25);
    } else {  
        vogel.rotate(45);
    }
    
    // Wenn die erste Barrier den X-Wert 400 erreicht, wird die zweite Barriere erstellt (nur für den ersten Durchlauf relevant)
    if (!reachMid && barriere1->getXCoord() == 400) {
        reachMid = true;
        barriere2 = erstelleBarriere();
    }
    
    // Wenn die Barrieren links am Bildschirm verschwinden, werden sie resettet
    if (barriere1->getXCoord() == -100) {
        delete barriere1;
        barriere1 = erstelleBarriere();
    } else if (barriere2 != nullptr && barriere2->getXCoord() == -100) {
        delete barriere2;
        barriere2 = erstelleBarriere();
    }
    
    // Bewegt die Barrieren
    barriere1->setXCoord(barriere1->getXCoord() - 10);
    if (barriere2 != nullptr) {
        barriere2->setXCoord(barriere2->getXCoord() - 10);
    }
    
    screenScore = Text("Score: " + to_string(score / 7), -20, 15, &spielfeld);
    
    iteration++;
}